In [1]:
import numpy as np

# Create monkey patches
np.float = float
np.int = int
np.object = object
np.bool = bool

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import gunshot_utils as utils
import importlib
import ast
import re
import os
import pickle
from glob import glob
import librosa.display
import IPython.display as ipd

import torch as th
import numpy as np
import torchaudio
from torch.utils.data import DataLoader
from IPython.display import Audio
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from pydub import AudioSegment
from pydub.playback import play

importlib.reload(utils)

In [3]:
# gunshot_df = pd.read_csv("/Users/borosabel/Documents/Uni/Thesis/PopMIR/Code/Audio/Gunshot/csv_combined/filtered_gunshot_metadata.csv")
# gunshot_audio_dir = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Audio/Gunshots/csv/edge-collected-gunshot-audio/edge-collected-gunshot-audio'

In [4]:
filtered_glock_df = pd.read_csv('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Code/Audio/Gunshot/on-the-fly/filtered_gunshot_metadata_glocks.csv')

In [5]:
music_df = pd.read_excel('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Excel/baseline_data_w_topics_w_features.xlsx', engine='openpyxl')

In [6]:
music_train_df, music_valid_df = train_test_split(music_df, test_size=0.2, random_state=42)
gunshot_train_df, gunshot_valid_df = train_test_split(filtered_glock_df, test_size=0.2, random_state=42)

In [7]:
class GunshotDetectionCNN(nn.Module):
    def __init__(self, num_frames):
        super(GunshotDetectionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=(3, 7))
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 1))
        self.conv2 = nn.Conv2d(10, 20, kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 1))

        dummy_input = th.zeros(1, 3, 80, num_frames)  # Shape: (batch_size, channels, height, width)
        dummy_output = self.pool2(F.relu(self.conv2(self.pool1(F.relu(self.conv1(dummy_input))))))
        output_size = dummy_output.view(-1).shape[0]

        self.fc1 = nn.Linear(output_size, 256)
        self.fc2 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        x = self.dropout(F.relu(self.fc1(x)))  # Apply dropout
        x = self.sigmoid(self.fc2(x))
        return x

# Example usage
model = GunshotDetectionCNN(num_frames=utils.NUM_FRAMES)

In [8]:
# music_paths = music_df['Path'].tolist()
# self.gunshot_paths = gunshot_df['filename'].tolist()
# self.gunshot_truth = gunshot_df['gunshot_location_in_seconds'].apply(
#     lambda x: utils.preprocess_gunshot_times(x, include_first_gunshot_only=True)
# ).tolist()
# 
# music_path = music_train_df.iloc[0]['Path']
# gunshot_path, gunshot_locations = gunshot_valid_df.iloc[0]['filename'], gunshot_valid_df.iloc[0]['gunshot_location_in_seconds']
# gunshot_locations = utils.preprocess_gunshot_times(gunshot_locations, include_first_gunshot_only=True)

In [9]:
# importlib.reload(utils)
# 
# utils.combine_music_and_gunshot(music_path, gunshot_path, gunshot_locations[0])

In [10]:
music_row = music_train_df.iloc[0]
gunshot_row = gunshot_train_df.iloc[0]

music_path, sr, = music_row['Path'], music_row['Sample Rate (Hz)']
gunshot_path, gunshot_locations, num_gunshots = gunshot_row['filename'], gunshot_row['gunshot_location_in_seconds'], gunshot_row['num_gunshots']

In [13]:
import torchaudio
import torch as th
import numpy as np
import random
from torch.utils.data import DataLoader

class GunshotDataset(th.utils.data.Dataset):
    def __init__(self, music_df, gunshot_df, excerpt_len=5.0, gunshot_placement_sec=2.0, gunshot_prob=1.0, min_db=3, max_db=5, max_non_gunshot_samples=1, mean=None, std=None):
        """
        :param music_df: DataFrame containing paths to music files.
        :param gunshot_df: DataFrame containing paths to gunshot files and timing info.
        :param excerpt_len: Length of the music segment in seconds.
        :param gunshot_placement_sec: Time in seconds where to place the gunshot in the music.
        :param gunshot_prob: Probability of adding a gunshot to the segment.
        :param min_db: Minimum gain (in dB) to apply to the gunshot.
        :param max_db: Maximum gain (in dB) to apply to the gunshot.
        :param max_non_gunshot_samples: Max number of non-gunshot samples to extract when no gunshots are present.
        """
        super().__init__()
        self.music_paths = music_df['Path'].tolist()
        self.gunshot_paths = gunshot_df['filename'].tolist()
        self.gunshot_truth = gunshot_df['gunshot_location_in_seconds'].apply(
            lambda x: utils.preprocess_gunshot_times(x, include_first_gunshot_only=True)
        ).tolist()
        print(self.gunshot_truth[0])
        self.excerpt_len = excerpt_len
        self.gunshot_placement_sec = gunshot_placement_sec
        self.gunshot_prob = gunshot_prob
        self.min_db = min_db
        self.max_db = max_db
        self.max_non_gunshot_samples = max_non_gunshot_samples

    def __getitem__(self, idx):
        fn_music = self.music_paths[idx]
        add_gunshot = (np.random.rand() < self.gunshot_prob)
        sample_rate = 44100

        if add_gunshot:
            gunshot_idx = np.random.randint(0, len(self.gunshot_paths) - 1)
            fn_gunshot = self.gunshot_paths[gunshot_idx]
            gunshot_times = self.gunshot_truth[gunshot_idx][0]

            # Combine music and gunshot
            music_segment, sr = utils.combine_music_and_gunshot(
                music_file=fn_music,
                gunshot_file=fn_gunshot,
                gunshot_time=gunshot_times,
                gunshot_volume_increase_dB=self.max_db,
                gunshot_placement_sec=self.gunshot_placement_sec,
                excerpt_len_sec=self.excerpt_len,
                sample_rate=utils.SAMPLING_RATE
            )
            label = 1
            spectrograms, labels = utils.preprocess_audio_train(music_segment, sr, label, gunshot_times)
        else:
            # Extract a segment of music without gunshots
            music_segment, sr = utils.extract_music_segment(
                music_file=fn_music,
                excerpt_len=self.excerpt_len,
                sample_rate=utils.SAMPLING_RATE
            )
            label = 0
            spectrograms, labels = utils.preprocess_audio_train(music_segment, sr, label)

        if not spectrograms or not labels:
            raise ValueError("Spectrograms or labels are empty after preprocessing")

        return spectrograms[0], labels[0]

    def __len__(self):
        return len(self.music_paths)

In [14]:
# Example usage
# Create training dataset
train_dataset = GunshotDataset(music_train_df, gunshot_train_df, excerpt_len=5.0, gunshot_placement_sec=2.0, min_db=5, max_db=10, gunshot_prob=0.5)

# Create validation dataset
valid_dataset = GunshotDataset(music_valid_df, gunshot_valid_df, excerpt_len=5.0, gunshot_placement_sec=2.0, min_db=5, max_db=10, gunshot_prob=0.5)

# Create DataLoader for training
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create DataLoader for validation (no need to shuffle validation data)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [109]:
import torch as th
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm  # Import tqdm for progress bars

device = th.device("cuda" if th.cuda.is_available() else "cpu")
use_cuda = th.cuda.is_available()

def train_model(model, optimizer, criterion, train_loader, valid_loader, epochs=10, mean=None, std=None, patience=3):
    if mean is None or std is None:
        raise ValueError("Mean and std must be provided for normalization.")

    mean = mean.to(device)
    std = std.to(device)
    model = model.to(device)
    best_score = 0.0
    epochs_since_improvement = 0

    if use_cuda:
        scaler = th.cuda.amp.GradScaler()  # For mixed precision training
    else:
        scaler = None  # No need for GradScaler on CPU

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        # Add tqdm progress bar for training loop
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}] Training")

        for features, labels in train_loader_tqdm:
            features, labels = features.to(device), labels.to(device).float().to(device)
            optimizer.zero_grad()
            features = (features - mean) / std

            if use_cuda:
                with th.cuda.amp.autocast():
                    outputs = model(features).view(-1)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(features).view(-1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * features.size(0)

            # Update tqdm description with current loss
            train_loader_tqdm.set_postfix(loss=loss.item())

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

        # Evaluate on the validation set
        model.eval()
        val_score = evaluate_model_simple(model, valid_loader, mean, std)

        if val_score > best_score:
            best_score = val_score
            epochs_since_improvement = 0
            print(f"New best ROC AUC score: {best_score:.4f}, model saved.")
            # Save the model if desired
            # torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_since_improvement += 1

        if epochs_since_improvement >= patience:
            print(f"No improvement in ROC AUC score for {patience} epochs. Stopping training.")
            break

    # After training, find the optimal threshold
    best_threshold = find_optimal_threshold_after_training(model, valid_loader, mean, std)

    # Compute and display the confusion matrix
    cm = compute_confusion_matrix(model, valid_loader, best_threshold, mean, std)
    display_confusion_matrix(cm)

    return best_threshold, best_score


def evaluate_model_simple(model, valid_loader, mean, std):
    """
    Evaluates the model on the validation set using ROC AUC score.

    Parameters:
        model (torch.nn.Module): The trained model.
        valid_loader (DataLoader): DataLoader for validation data.
        mean (torch.Tensor): Mean for normalization.
        std (torch.Tensor): Standard deviation for normalization.

    Returns:
        auc (float): ROC AUC score on the validation set.
    """
    all_outputs = []
    all_labels = []

    # Add tqdm progress bar for validation loop
    valid_loader_tqdm = tqdm(valid_loader, desc="Validation")

    with th.no_grad():
        for features, labels in valid_loader_tqdm:
            features = features.to(device)
            labels = labels.to(device).float()
            features = (features - mean) / std
            outputs = model(features).view(-1).cpu().numpy()
            all_outputs.append(outputs)
            all_labels.append(labels.cpu().numpy())

    all_outputs = np.concatenate(all_outputs)
    all_labels = np.concatenate(all_labels)
    auc = roc_auc_score(all_labels, all_outputs)
    print(f"Validation ROC AUC: {auc:.4f}")
    return auc

def find_optimal_threshold_after_training(model, valid_loader, mean, std):
    """
    Finds the optimal threshold after training using ROC curve and Youden's J statistic.

    Parameters:
        model (torch.nn.Module): The trained model.
        valid_loader (DataLoader): DataLoader for validation data.
        mean (torch.Tensor): Mean for normalization.
        std (torch.Tensor): Standard deviation for normalization.

    Returns:
        optimal_threshold (float): Optimal threshold value.
    """
    all_outputs = []
    all_labels = []

    # Add tqdm progress bar for validation loop
    valid_loader_tqdm = tqdm(valid_loader, desc="Finding Optimal Threshold")

    with th.no_grad():
        for features, labels in valid_loader_tqdm:
            features = features.to(device)
            labels = labels.to(device).float()
            features = (features - mean) / std
            outputs = model(features).view(-1).cpu().numpy()
            all_outputs.append(outputs)
            all_labels.append(labels.cpu().numpy())

    all_outputs = np.concatenate(all_outputs)
    all_labels = np.concatenate(all_labels)

    fpr, tpr, thresholds = roc_curve(all_labels, all_outputs)
    youdens_j = tpr - fpr
    idx = np.argmax(youdens_j)
    optimal_threshold = thresholds[idx]

    print(f"Optimal threshold found: {optimal_threshold:.4f}")
    return optimal_threshold

def compute_confusion_matrix(model, valid_loader, threshold, mean, std):
    """
    Compute confusion matrix using batch processing.

    Parameters:
        model (torch.nn.Module): The trained model.
        valid_loader (DataLoader): DataLoader for validation data.
        threshold (float): Threshold to convert probabilities to binary predictions.
        mean (torch.Tensor): Mean for normalization.
        std (torch.Tensor): Standard deviation for normalization.

    Returns:
        cm (numpy.ndarray): Confusion matrix.
    """
    all_outputs = []
    all_labels = []

    # Add tqdm progress bar for validation loop
    valid_loader_tqdm = tqdm(valid_loader, desc="Computing Confusion Matrix")

    with th.no_grad():
        for features, labels in valid_loader_tqdm:
            features = features.to(device)
            labels = labels.cpu().numpy()
            features = (features - mean) / std
            outputs = model(features).view(-1).cpu().numpy()
            all_outputs.append(outputs)
            all_labels.append(labels)

    all_outputs = np.concatenate(all_outputs)
    all_labels = np.concatenate(all_labels)

    predictions = (all_outputs >= threshold).astype(int)
    cm = confusion_matrix(all_labels, predictions)

    return cm

def display_confusion_matrix(cm):
    """
    Displays the confusion matrix using matplotlib.

    Parameters:
        cm (numpy.ndarray): Confusion matrix.
    """
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
    disp.plot(cmap='magma')
    plt.title('Confusion Matrix')
    plt.show()


In [110]:
from tqdm import tqdm

def compute_mean_std(dataloader):
    """Compute mean and std of the entire dataset with progress tracking using tqdm."""
    # Use tqdm to wrap the dataloader and show progress
    l = []
    for features, _ in tqdm(dataloader, desc="Computing mean and std"):
        l += features
    tmp = th.cat(l)
    mean = th.mean(tmp, dim=(0, 2)).unsqueeze(1)
    std = th.std(tmp, dim=(0, 2)).unsqueeze(1)
    return mean, std

mean, std = compute_mean_std(train_loader)

# Move mean and std to the device for training
mean = mean.to(device) 
std = std.to(device)

In [111]:
epochs = 2
lr = 3e-4

optimizer = th.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = th.nn.BCELoss()

# Train the model
best_threshold, best_score = train_model(
    model, optimizer, criterion, train_loader, valid_loader,
    epochs=10, mean=mean, std=std, patience=3
)

print(f"Training completed. Best ROC AUC: {best_score:.4f}, Optimal Threshold: {best_threshold:.4f}")

In [114]:
def manual_evaluate_test(model, feature, threshold, frame_size=utils.NUM_FRAMES, sampling_rate=utils.SAMPLING_RATE, hop_length=utils.HOP_LENGTH, mean=None, std=None, step_size=None, filter_time_sec=1):
    """
    Manually evaluate the model on an audio feature, returning time positions where gunshots are detected.

    Parameters:
        model: The trained model.
        feature: The feature (e.g., spectrogram) to evaluate.
        threshold: The prediction threshold for gunshots.
        frame_size: Number of frames to use in each evaluation.
        sampling_rate: Audio sampling rate.
        hop_length: Hop length in samples for each frame.
        mean: Mean for normalization.
        std: Standard deviation for normalization.
        step_size: Step size for moving through frames (default: frame_size // 2).
        filter_time_sec: Time (in seconds) to filter out close consecutive predictions.
    
    Returns:
        List of tuples (minutes, seconds, output) where gunshots are detected along with the model's output.
    """
    if mean is None or std is None:
        raise ValueError("Mean and std must be provided for normalization.")

    mean = mean.to(device)
    std = std.to(device)
    model = model.to(device)
    model.eval()

    predictions = []

    # Normalize feature
    feature = feature.to(device)
    feature = (feature - mean) / std

    num_frames = feature.shape[2]

    # If step_size is not specified, default to half the frame size
    if step_size is None:
        # step_size = frame_size // 2  # Adjust step_size if necessary
        step_size = 1

    total_iterations = 0  # To count the iterations

    with th.no_grad():
        # Loop through overlapping frames with smaller step size
        for j in range(0, num_frames - frame_size + 1, step_size):
            total_iterations += 1
            start = j
            end = j + frame_size

            input_frame = feature[:, :, start:end].unsqueeze(0).float()
            output = model(input_frame).squeeze().item()
            predictions.append((output, start))  # Keep track of output and start position

        return predictions

        # Sort predictions by the time index
        res = []
        for output, start in predictions:
            if output >= threshold:
                time_in_seconds = start * hop_length / sampling_rate
                minutes = int(time_in_seconds // 60)
                seconds = time_in_seconds % 60
                res.append((minutes, seconds, time_in_seconds, output))  # Add output to the result

    # Filter out close consecutive detections
    filtered_res = []
    last_detection_time = -float('inf')  # Initialize with negative infinity to accept the first detection

    for minutes, seconds, time_in_seconds, output in res:
        if time_in_seconds - last_detection_time >= filter_time_sec:
            # Append with output value included
            filtered_res.append((minutes, seconds, output))
            last_detection_time = time_in_seconds  # Update the last detected time

    # Return results with raw model output for comparison
    return filtered_res

In [112]:
spectrograms, sample_rates = utils.preprocess_audio(['/Users/borosabel/Documents/Uni/Thesis/PopMIR/M.I.A. - Paper Planes.mp3'])

In [115]:
predictions = manual_evaluate_test(model, spectrograms, threshold=best_threshold, mean=mean, std=std, step_size=1,
                                   filter_time_sec=1.5)